In [1]:
import numpy as np

In [2]:
data_ibl = np.load("20240204_1426.npy")
data_1 = np.load("05022024_1.npy")
data_2 = np.load("05022024_2.npy")

In [3]:
def correct_time(ibl_time):
    n = 0
    delta = 0
    for i in range(len(ibl_time)-1):
        if abs(ibl_time[i+1]-ibl_time[i])>2000:
            n += 1
            ibl_time[(i+1):] += (ibl_time[i] - delta)
            delta += (ibl_time[i] - delta)
    print(n, " раз переполнилось время на ибл")

In [4]:
def find_points1(omega1, omega2, n_peaks=10, limit=500):
    counter = 0
    in_peak = False
    same_time = []
    index2 = 0 

    for i in range(len(omega1)):
        if abs(omega1[i]) > limit and not in_peak:
            in_peak = True
            counter += 1
            for j in range(index2, len(omega2), 1):
                if abs(omega2[j]) > limit:
                    same_time.append((i,j))
                    index2 = j
                    break
            
        elif in_peak and abs(omega1[i]) < limit:
            in_peak = False
            for j in range(index2, len(omega2), 1):
                if abs(omega2[j]) < limit:
                    same_time.append((i,j))
                    index2 = j
                    break
            if counter == n_peaks:
                break
    if counter < n_peaks:
        print("Only ", counter, " peaks in data")
    else:
        return same_time

In [5]:
def find_points2(omegaX, omegaZ, omega2, n_peaks=10, limit=600):
    counter = 0
    in_peak = False
    same_time = []
    index1 = 0 
    indexX = 0
    indexZ = 0

    for i in range(len(omega2)):
        if abs(omega2[i]) > limit and not in_peak:
            in_peak = True
            counter += 1
            for j in range(index1, len(omegaX), 1):
                if abs(omegaX[j]) > limit:
                    indexX = j
                    break
            for k in range(index1, len(omegaZ), 1):
                if abs(omegaZ[k]) > limit:
                    indexZ = k
                    break
            same_time.append((i, min(indexX, indexZ)))
            index1 = min(indexX, indexZ)
            
        elif in_peak and abs(omega2[i]) < limit:
            in_peak = False
            if indexX < indexZ:
                for j in range(index1, len(omegaX), 1):
                    if abs(omegaX[j]) < limit:
                        same_time.append((i,j))
                        index1 = j
                        break
            if indexX > indexZ:
                for k in range(index1, len(omegaZ), 1):
                    if abs(omegaZ[k]) < limit:
                        same_time.append((i,k))
                        index1 = k
                        break
            if counter == n_peaks:
                break
                
    if counter < n_peaks:
        print("Only ", counter, " peaks in data")
    else:
        return same_time

In [6]:
def concatenate_axes(time1, time2, fi1, fi2, a1, b1, a2, b2):
    time_1_2ibl = a1*time1 + b1
    time_2_2ibl = a2*time2 + b2

    time12 = np.empty(time1.size + time2.size, dtype=time1.dtype)
    fi1_12 = np.empty(time1.size + time2.size, dtype=time1.dtype) 
    fi2_12 = np.empty(time1.size + time2.size, dtype=time1.dtype)
    i, j, k = 0, 0, 0

    while i < time1.size and j < time2.size:
        if time1[i] < time2[j]:
            time12[k] = time1[i]
            fi1_12[k] = fi1[i]
            fi2_12[k] = 0 if k==0 else fi2_12[k-1]
            i += 1
        else:
            time12[k] = time2[j]
            fi2_12[k] = fi2[j]
            fi1_12[k] = 0 if k==0 else fi1_12[k-1]
            j += 1
        k += 1

    while i < time1.size:
        time12[k] = time1[i]
        fi1_12[k] = fi1[i]
        fi2_12[k] = fi2_12[k-1]
        i += 1
        k += 1

    while j < time2.size:
        time12[k] = time2[j]
        fi2_12[k] = fi2[j]
        fi1_12[k] = fi1_12[k-1]
        j += 1
        k += 1
    
    return time12, fi1_12, fi2_12

In [7]:
 def print_reference(time12, fi1, fi2, begin_koord=np.array([[1,0,0],[0,-1,0],[0,0,-1]])):
    OmegaX = np.array([])
    OmegaY = np.array([])
    OmegaZ = np.array([])

    M12 = np.array([])
    ex = begin_koord[:, 0]
    ey = begin_koord[:, 1]
    ez = begin_koord[:, 2]
    fi10 = None
    fi1  = None
    fi20 = None
    fi2  = None

    Omega = np.array([0, 12.44, 8.42])

    file = open('etalon.txt', 'w')
    file.write("{:8}".format("time") + "{:15}".format("omega_x,gph") + "{:15}".format("omega_y,gph") + "{:15}".format("omega_z,gph") + '\n')
    for i in range(1, len(time12)//100 +1):
        fi10, fi1 = fi1[i-1], fi1[i]
        fi20, fi2 = fi2[i-1], fi2[i]
        y, z = -ey[1], -ey[2]
        dfi1 = math.radians(fi1-fi10)
        dfi2 = math.radians(fi2-fi20)
        M12 = np.array([[np.cos(dfi1),                                 -(y*np.sin(dfi2)+z*np.cos(dfi2))*np.sin(dfi1),                                  (y*np.cos(dfi2)-z*np.sin(dfi2))*np.sin(dfi1)],
                        [(y*np.sin(dfi2)+z*np.cos(dfi2))*np.sin(dfi1), np.cos(dfi1)+(1-np.cos(dfi1))(y*np.cos(dfi2)-z*np.sin(dfi2))**2,                (1-np.cos(dfi1))(y*np.cos(dfi2)-z*np.sin(dfi2))(y*np.sin(dfi2)+z*np.cos(dfi2))], 
                        [(z*np.sin(dfi2)-y*np.cos(dfi2))*np.sin(dfi1), (1-np.cos(dfi1))(y*np.sin(dfi2)+z*np.cos(dfi2))(y*np.cos(dfi2)-z*np.sin(dfi2)), np.cos(dfi1)+(1-np.cos(dfi1))(y*np.sin(dfi2)+z*np.cos(dfi2))**2]])
        ex = M12 @ ex
        ey = M12 @ ey
        ez = M12 @ ez
        file.write("{:8.3f}".format(time12[i]) + "{:15.8f}".format(np.dot(ex, Omega)) + "{:15.8f}".format(np.dot(ey, Omega)) + "{:15.8f}".format(np.dot(ez, Omega)) + '\n')
    file.close()

In [8]:
def build_reference(data_ibl, data_1, data_2, n_peaks=10, limit1=500, limit2=600):
    time_ibl = data_ibl[:, 0]
    omegaX_ibl = data_ibl[:, 1]
    omegaY_ibl = data_ibl[:, 2] * (-1)
    omegaZ_ibl = data_ibl[:, 3]
    time_1 = data_1[:, 0]
    fi_1 = data_1[:, 1]
    theta_1 = data_1[:, 2]
    omega_1 = theta_1*3600
    time_2 = data_2[:, 0]
    fi_2 = data_2[:, 1]
    theta_2 = data_2[:, 2]
    omega_2 = theta_2*3600
    
    correct_time(time_ibl)
    
    points1 = find_points1(omegaY_ibl, omega_1, n_peaks, limit1)
    points2 = find_points2(omegaX_ibl, omegaZ_ibl, omega_2, n_peaks, limit2)
    
    timeibl1= np.array([])
    time1 = np.array([])
    for i in range(len(points1)):
        timeibl1 = np.append(timeibl1, time_ibl[points1[i][0]])
        time1 = np.append(time1, time_1[points1[i][1]])
    
    timeibl2 = np.array([])
    time2 = np.array([])
    for i in range(len(points2)):
        timeibl2 = np.append(timeibl2, time_ibl[points2[i][1]])
        time2 = np.append(time2, time_2[points2[i][0]])
        
    a1, b1 = np.polyfit(time1, timeibl1, 1)
    a2, b2 = np.polyfit(time2, timeibl2, 1)
    
    time12, fi1_12, fi2_12 = concatenate_axes(time1, time2, fi_1, fi_2, a1, b1, a2, b2)
    print(len(time12))
    print_reference(time12, fi1_12, fi2_12)

In [9]:
build_reference(data_ibl, data_1, data_2)

14  раз переполнилось время на ибл
40
